In [ ]:
import pandas as pd
import warnings
import numpy as np
from sqlalchemy import create_engine, Table, MetaData
import urllib.parse
from datetime import datetime
from sqlalchemy.sql import text
import re
import json
import pyodbc
import warnings
import requests
import schedule
from apscheduler.schedulers.blocking import BlockingScheduler
import pandas as pd
from io import BytesIO
import os
import tempfile
import pythoncom
import win32com.client
import time


#                                                                             multi incentive 
    
username = 'prashant'
password = 'CredEnt!@L@2025!$'
server = '172.16.1.13'
database = 'Ops_Analytics'
driver = 'ODBC Driver 17 for SQL Server'

quoted_password = urllib.parse.quote_plus(password)
connection_string = f"mssql+pyodbc://{username}:{quoted_password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)
conn = create_engine(connection_string)

sql_script = """
-- Drop temp tables if they exist
DROP TABLE IF EXISTS #new_table;
DROP TABLE IF EXISTS #MultiINCENTIVE;
DROP TABLE IF EXISTS #FormFill_1;
-- Drop the temporary table if it exists
DROP TABLE IF EXISTS #new_table;
-- DIALLER CDR
SELECT 
    date,
    End_stamp,
    Client_number,
    Disposition_name,
    Sub_Disposition_Name,
    Answered_Seconds,
    Campaign_name,
    agent_name
INTO #new_table
FROM RUNO.[dbo].[CDR_new_dialer]
WHERE Campaign_name IN ( 'multi google','multi rejected')
  AND date >= '2025-07-01';
-- Drop the temporary table if it exists
DROP TABLE IF EXISTS #FormFill_1;
-- LEAD TABLE
select * 
INTO #FormFill_1
from
(
SELECT *,
       CASE WHEN IsLeadCreated = 1 THEN 1 ELSE 0 END AS formfill,
       ROW_NUMBER() OVER (PARTITION BY right(mobile,10) ORDER BY CreatedOn DESC) AS rank_1
FROM (
    SELECT 
        mobile,
        LeadId,
        IsDedupe,
        IsLeadCreated,
        CreatedOn,
        SubLenderId
    FROM awssql.[multilender-v2].dbo.Leads WITH (NOLOCK)
    WHERE TRY_CAST(CreatedOn AS DATE) >= '2025-04-01'
	and SubLenderId = 16
    UNION ALL
    SELECT 
        mobile,
        LeadId,
        IsDedupe,
        IsLeadCreated,
        CreatedOn,
        SubLenderId
    FROM awssql.[multilender-v2].dbo.Leads_Arch WITH (NOLOCK)
    where TRY_CAST(CreatedOn AS DATE) >= '2025-04-01'
	and SubLenderId = 16
) kp
where   SubLenderId IN (7,16)
) c where c.rank_1 = 1
-- DIALER CALLING REPORT
SELECT *
into #MultiINCENTIVE
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY mobile ORDER BY End_stamp DESC) AS end_1
    FROM #FormFill_1 apl
    LEFT JOIN (
        SELECT *
        FROM #new_table
        WHERE agent_name NOT IN ('None')
    ) tpl
    ON apl.mobile = RIGHT(tpl.client_number, 10)
    AND TRY_CAST(apl.CreatedOn AS DATE) >= TRY_CAST(tpl.End_stamp AS DATE)
) kop
WHERE end_1 = 1
and Date IS NOT null

"""

with engine.begin() as connection:
    connection.execute(text(sql_script))

query = "SELECT * from #MultiINCENTIVE"
df_multi = pd.read_sql(query, engine)
df_multi_mail = df_multi[["mobile","End_stamp","Disposition_name","agent_name"]]  

import os, time, tempfile, pythoncom, win32com.client
from io import BytesIO
import pandas as pd

buffer = BytesIO()
with pd.ExcelWriter(buffer, engine='openpyxl') as writer:
    df_multi_mail.to_excel(writer, index=False)

buffer.seek(0)
tmp_path = None

try:
    with tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx') as tmp:
        tmp.write(buffer.read())
        tmp_path = tmp.name

    pythoncom.CoInitialize()
    outlook = win32com.client.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0)
    # mail.To = 'kunal.sharma@wecredit.co.in'
    mail.To = 'sushmita.samal@wecredit.co.in'
    mail.Subject = 'MULTI INCENTIVE'
    mail.Body = 'Hi,\n\nToday\'s TODAY INCENTIVE.\n\nThanks.'
    mail.Attachments.Add(tmp_path)
    mail.Send()
    print(f"✅ Email sent successfully. Temporary file used: {tmp_path}")

except Exception as e:
    print(f"❌ Failed to send email: {e}")

finally:
    if tmp_path and os.path.exists(tmp_path):
        os.unlink(tmp_path)

today_date = datetime.now().strftime('%Y-%m-%d')
# df_multi_mail.to_excel("multi{today_date}.xlsx",index= False)
df_multi_mail.to_excel(f'multi{today_date}.xlsx', index=False, header=True)



#                                                              insta money
    
username = 'prashant'
password = 'CredEnt!@L@2025!$'
server = '172.16.1.13'
database = 'Ops_Analytics'
driver = 'ODBC Driver 17 for SQL Server'

quoted_password = urllib.parse.quote_plus(password)
connection_string = f"mssql+pyodbc://{username}:{quoted_password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)
conn = create_engine(connection_string)

sql_script = """
WITH joined_data AS (
    SELECT 
        TRY_CAST(mis.sanctioned_amount AS FLOAT) AS sanctioned_amount,
		cdr.Disposition_Name,
        cdr.Client_Number,
        cdr.Agent_Name,
        cdr.End_Stamp,
        DATEDIFF(DAY, TRY_CAST(mis.disbursement_date AS DATE), cdr.End_Stamp) AS diff_days,
        ROW_NUMBER() OVER (PARTITION BY cdr.Client_Number ORDER BY cdr.End_Stamp DESC) AS rn
    FROM mis.dbo.lenden_automation_mis_new AS mis
    INNER JOIN RUNO.dbo.Dialer_CDR_report AS cdr
        ON mis.mobile_number = RIGHT(cdr.Client_Number, 10)
    WHERE 
		mis.disbursement_date is not null
        AND mis.disbursement_date >= '2025-08-01'
        AND cdr.Campaign_Name = 'Insta Money'
)
select * FROM joined_data where rn = 1 and Agent_Name != 'None'

"""
df_multi = pd.read_sql(sql_script, engine)
df_multi_mail = df_multi[["sanctioned_amount","Disposition_Name","Client_Number","Agent_Name","End_Stamp"]]  

import os, time, tempfile, pythoncom, win32com.client
from io import BytesIO
import pandas as pd

buffer = BytesIO()
with pd.ExcelWriter(buffer, engine='openpyxl') as writer:
    df_multi_mail.to_excel(writer, index=False)

buffer.seek(0)
tmp_path = None

try:
    with tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx') as tmp:
        tmp.write(buffer.read())
        tmp_path = tmp.name

    pythoncom.CoInitialize()
    outlook = win32com.client.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0)
    # mail.To = 'kunal.sharma@wecredit.co.in'
    mail.To = 'sushmita.samal@wecredit.co.in'
    mail.Subject = 'INSTA MONEY'
    mail.Body = 'Hi,\n\nToday\'s TODAY INCENTIVE.\n\nThanks.'
    mail.Attachments.Add(tmp_path)
    mail.Send()
    print(f"✅ Email sent successfully. Temporary file used: {tmp_path}")

except Exception as e:
    print(f"❌ Failed to send email: {e}")

finally:
    if tmp_path and os.path.exists(tmp_path):
        os.unlink(tmp_path)

today_date = datetime.now().strftime('%Y-%m-%d')
# df_multi_mail.to_excel("multi{today_date}.xlsx",index= False)


# scheduler = BlockingScheduler()
# scheduler.add_job(run_task, 'cron', hour=9, minute=15)
# # scheduler.add_job(run_task, 'interval', seconds=10)
# # scheduler.add_job(run_task, 'cron', hour="9-18", minute=0)
# scheduler.start()


import os, time, tempfile, pythoncom, win32com.client
from io import BytesIO
import pandas as pd

buffer = BytesIO()
with pd.ExcelWriter(buffer, engine='openpyxl') as writer:
    df_multi_mail.to_excel(writer, index=False)

buffer.seek(0)
tmp_path = None

try:
    with tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx') as tmp:
        tmp.write(buffer.read())
        tmp_path = tmp.name

    pythoncom.CoInitialize()
    outlook = win32com.client.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0)
    # mail.To = 'kunal.sharma@wecredit.co.in'
    mail.To = 'kanhaiya.mali@wecredit.co.in'
    mail.Subject = 'INSTA MONEY'
    mail.Body = 'Hi,\n\nToday\'s TODAY INCENTIVE.\n\nThanks.'
    mail.Attachments.Add(tmp_path)
    mail.Send()
    print(f"✅ Email sent successfully. Temporary file used: {tmp_path}")

except Exception as e:
    print(f"❌ Failed to send email: {e}")

finally:
    if tmp_path and os.path.exists(tmp_path):
        os.unlink(tmp_path)

today_date = datetime.now().strftime('%Y-%m-%d')
# df_multi_mail.to_excel("multi{today_date}.xlsx",index= False)



import os, time, tempfile, pythoncom, win32com.client
from io import BytesIO
import pandas as pd

buffer = BytesIO()
with pd.ExcelWriter(buffer, engine='openpyxl') as writer:
    df_multi_mail.to_excel(writer, index=False)

buffer.seek(0)
tmp_path = None

try:
    with tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx') as tmp:
        tmp.write(buffer.read())
        tmp_path = tmp.name

    pythoncom.CoInitialize()
    outlook = win32com.client.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0)
    # mail.To = 'kunal.sharma@wecredit.co.in'
    mail.To = 'vishakha.mishra@wecredit.co.in'
    mail.Subject = 'INSTA MONEY'
    mail.Body = 'Hi,\n\nToday\'s TODAY INCENTIVE.\n\nThanks.'
    mail.Attachments.Add(tmp_path)
    mail.Send()
    print(f"✅ Email sent successfully. Temporary file used: {tmp_path}")

except Exception as e:
    print(f"❌ Failed to send email: {e}")

finally:
    if tmp_path and os.path.exists(tmp_path):
        os.unlink(tmp_path)

today_date = datetime.now().strftime('%Y-%m-%d')
# df_multi_mail.to_excel("multi{today_date}.xlsx",index= False)
df_multi_mail.to_excel(f'INSTAMONEY{today_date}.xlsx', index=False, header=True)


#                                                                      mv
username = 'prashant'
password = 'CredEnt!@L@2025!$'
server = '172.16.1.13'
database = 'Ops_Analytics'
driver = 'ODBC Driver 17 for SQL Server'

quoted_password = urllib.parse.quote_plus(password)
connection_string = f"mssql+pyodbc://{username}:{quoted_password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)
conn = create_engine(connection_string)

sql_script = """
WITH base_data AS (
    SELECT 
        mis.phone_number,
        mis.current_status,
		mis.loan_date,
        cdr.end_stamp,
        cdr.Disposition_Name,
        cdr.Agent_Name,
        DATEDIFF(DAY,  cdr.end_stamp,mis.loan_date) AS dtdiff
    FROM (
        SELECT 
            CAST(phone_number AS VARCHAR(20)) AS phone_number,
            disbursal_date AS loan_date,
            current_status
        FROM Ops_Analytics.dbo.MV_MIS_PK
        WHERE 
            current_status = '11. Disbursed'
            AND date_created >= DATEADD(DAY, -30, GETDATE()) 
            AND disbursal_date IS NOT NULL
            AND disbursal_date >= '2025-08-01'
    ) mis
    LEFT JOIN (
        -- First Source: Dialer_CDR_report
        SELECT 
            RIGHT(CAST(Client_number AS VARCHAR(20)), 10) AS phone_number,
            Disposition_Name,
            Agent_Name,
            end_stamp
        FROM (
            SELECT *,
                   RANK() OVER (PARTITION BY Client_number ORDER BY end_stamp DESC) AS rk
            FROM RUNO.dbo.Dialer_CDR_report WITH (NOLOCK)
            WHERE 
                campaign_name IN ('Moneyview dis', 'Money view repeat')
                AND TRY_CAST(end_stamp AS DATE) >= DATEADD(DAY, -40, CAST(GETDATE() AS DATE))
                AND Agent_Name NOT IN ('none')
        ) AS RankedData
        WHERE rk = 1

        UNION ALL

        -- Second Source: MV - INTERACTIONS ALL
        SELECT 
            RIGHT(CAST(Phone AS VARCHAR(20)), 10) AS phone_number,
            Status AS Disposition_Name,
            [Assigned to] AS Agent_Name,
            [Interaction Date] AS end_stamp
        FROM (
            SELECT *,
                   RANK() OVER (PARTITION BY Phone ORDER BY [Interaction Date] DESC) AS rk
            FROM [RUNO].[dbo].[MV - INTERACTIONS ALL]
            WHERE 
                TRY_CAST([Interaction Date] AS DATETIME) >= DATEADD(DAY, -40, GETDATE())
                AND [Assigned to] IS NOT NULL
        ) AS InteractionRanked
        WHERE rk = 1
    ) cdr
        ON mis.phone_number = cdr.phone_number
    WHERE DATEDIFF(DAY, mis.loan_date, cdr.end_stamp) IN (1, 2, 3, 0, -1)
),
ranked_data AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY phone_number ORDER BY end_stamp DESC) AS rn
    FROM base_data
)
SELECT *
    phone_number,
    current_status,
    end_stamp,
    Disposition_Name,
    Agent_Name
FROM ranked_data
WHERE rn = 1
and Disposition_Name not in ('Final Not Interested')
order by End_Stamp


"""
df_multi = pd.read_sql(sql_script, engine)
df_multi_mail = df_multi[["sanctioned_amount","Disposition_Name","Client_Number","Agent_Name","End_Stamp"]]  

import os, time, tempfile, pythoncom, win32com.client
from io import BytesIO
import pandas as pd

buffer = BytesIO()
with pd.ExcelWriter(buffer, engine='openpyxl') as writer:
    df_multi_mail.to_excel(writer, index=False)

buffer.seek(0)
tmp_path = None

try:
    with tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx') as tmp:
        tmp.write(buffer.read())
        tmp_path = tmp.name

    pythoncom.CoInitialize()
    outlook = win32com.client.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0)
    mail.To = 'kanhaiya.mali@wecredit.co.in'
    mail.Subject = 'MV  Incentive data '
    mail.Body = 'Hi,\n\nToday\'s TODAY INCENTIVE.\n\nThanks.'
    mail.Attachments.Add(tmp_path)
    mail.Send()
    print(f"✅ Email sent successfully. Temporary file used: {tmp_path}")

except Exception as e:
    print(f"❌ Failed to send email: {e}")

finally:
    if tmp_path and os.path.exists(tmp_path):
        os.unlink(tmp_path)

today_date = datetime.now().strftime('%Y-%m-%d')
# df_multi_mail.to_excel("multi{today_date}.xlsx",index= False)
df_multi_mail.to_excel(f'MV{today_date}.xlsx', index=False, header=True)